In [1]:
import ROOT

OBJ: TStyle	ildStyle	ILD Style : 0 at: 0x3d2d930
Welcome to JupyROOT 6.28/10


In [2]:
%jsroot on

In [3]:
ROOT.ildStyle.SetOptStat(1)

In [4]:
%%cpp
using namespace ROOT::VecOps;

In [5]:
df = ROOT.RDataFrame("events", ("data/truejet/test/sw_sl/eLpL_truejet.edm4hep.root", "data/truejet/test/sw_sl/eLpR_truejet.edm4hep.root"))
# df = ROOT.RDataFrame("events", "data/truejet/test/sw_sl/eLpL_truejet.edm4hep.root")
# df = ROOT.RDataFrame("events", "data/truejet/test/sw_sl/eLpR_truejet.edm4hep.root")

In [6]:
# df = df.Range(100)

In [7]:
%%cpp
//auto bPol_e = [](unsigned int slot, const ROOT::RDF::RSampleInfo &id) { return id.AsString(); };
auto bPol_e = [](unsigned int slot, const ROOT::RDF::RSampleInfo &id) { return id.Contains("eL") ? -1 : 1; };
auto bPol_p = [](unsigned int slot, const ROOT::RDF::RSampleInfo &id) { return id.Contains("pL") ? -1 : 1; };

In [8]:
df = df.DefinePerSample("beamPol_e", "bPol_e(rdfslot_, rdfsampleinfo_)")
df = df.DefinePerSample("beamPol_p", "bPol_p(rdfslot_, rdfsampleinfo_)")

In [9]:
# let's start with the InitialColourNeutrals as they have set energy and momentum (and mass, I hope it's consistent ;))

# rely on the relevant particle ID being the first one
# returns a vector containing the idx of the first PID in the ICN_PID collection belonging to the ICN
df = df.Define("pid_idx", "InitialColourNeutrals.particleIDs_begin")

# get the masks for the qq and lnu InitialColourNeutrals (W's)
# for each pid idx take the ICN_PID type and check if it is 1 (quarks) or 2 (lepton)
# returns a vector of ICN.size() with a 1 at the corresponding position, can be used to access all ICN.something fields
df = df.Define("qq_ICN_mask", "Take(InitialColourNeutrals_particleIDs.type, pid_idx) == 1")
df = df.Define("lnu_ICN_mask", "Take(InitialColourNeutrals_particleIDs.type, pid_idx) == 2")

# get the masses of the InitialColourNeutrals (inv. mass of the lnu and qqbar systems)
df = df.Define("m_qq", "InitialColourNeutrals.mass[qq_ICN_mask]")
df = df.Define("m_lnu", "InitialColourNeutrals.mass[lnu_ICN_mask]")

# There is only one lnu ICN so for some things its easier to use its index as
# RVec[mask] -> RVec, but RVec[idx] -> single element
df = df.Define("lnu_ICN_idx", "ArgMax(lnu_ICN_mask)")

# should be able to get e, nu, q, qbar + gluon jets by looking at which truejets belong to the INC
# I only need the separate q when I also try to do quark charge stuff so for now I am happy with qq ICN
# returns a vector of size two with the indices of l and nu in the truejet collection
df = df.Define("lnu_TJ_idx", "Range(InitialColourNeutrals.particles_begin[lnu_ICN_idx], InitialColourNeutrals.particles_end[lnu_ICN_idx])")

# TrueJets and TrueJets_particleIDs have the same number of entries and in the same order (this assumption should be safe)
# Take the TJ_PID.PDGs and check if e or nu
# returns vector of TrueJets.size() with a 1 where the l is
# FIXME: need to change hardcoded comparison for other l than e
df = df.Define("l_TJ_idxs_mask", "abs(Take(TrueJets_particleIDs.PDG, lnu_TJ_idx)) == 11")
df = df.Define("nu_TJ_idxs_mask", "abs(Take(TrueJets_particleIDs.PDG, lnu_TJ_idx)) == 12")

# As we only deal with exactly _one_ l and _one_ nu it might be easier to use the idx instead of the mask as
# RVec[mask] -> RVec, but RVec[idx] -> single element
# so that we don't need to worry about the additional vector layer
# the idx is the only 1 in the mask, the rest is 0 -> just use ArgMax
df = df.Define("l_TJ_idx", "lnu_TJ_idx[ArgMax(l_TJ_idxs_mask)]")
df = df.Define("nu_TJ_idx", "lnu_TJ_idx[ArgMax(nu_TJ_idxs_mask)]")


In [10]:
# FIXME: debug stuff
df.Range(17).Display(("lnu_ICN_idx", "lnu_TJ_idx", "l_TJ_idxs_mask", "nu_TJ_idxs_mask", "l_TJ_idx", "nu_TJ_idx"), 17, 500).Print()
# foo = df.Range(1).Take["RVec<std::size_t>"]("lnu_TJ_idx").GetValue()

+-----+-------------+------------+----------------+-----------------+----------+-----------+
| Row | lnu_ICN_idx | lnu_TJ_idx | l_TJ_idxs_mask | nu_TJ_idxs_mask | l_TJ_idx | nu_TJ_idx | 
+-----+-------------+------------+----------------+-----------------+----------+-----------+
| 0   | 1           | 2          | 1              | 0               | 2        | 3         | 
|     |             | 3          | 0              | 1               |          |           | 
+-----+-------------+------------+----------------+-----------------+----------+-----------+
| 1   | 1           | 2          | 1              | 0               | 2        | 3         | 
|     |             | 3          | 0              | 1               |          |           | 
+-----+-------------+------------+----------------+-----------------+----------+-----------+
| 2   | 1           | 2          | 1              | 0               | 2        | 3         | 
|     |             | 3          | 0              | 1           

In [11]:
# FIXME: debug stuff
# print(foo)
# print(foo[0])

In [12]:
# Unfortunately, none of the kinematics of the true jets are set (maybe just because we are doing a generator level thing here)
# We still need to go through the TrueJetMCParticleLink where rec are the TrueJets and sim are the MCParticles
# get mask for the TrueJetMCParticleLinks pointing to our truejet l
df = df.Define("l_TJ2MC_mask", "_TrueJetMCParticleLink_rec.index == l_TJ_idx")
# Get indices of all MCParticles belonging to the links
df = df.Define("l_MC_idxs", "_TrueJetMCParticleLink_sim.index[l_TJ2MC_mask]")
# There will be multiple results, containing the l in various stages and possibly FSR gammas
# We take the only l with genstat 1 -> the one also used in the detector simulation
# TODO: this is a choice! Figure out if it is the one we want to make
# XXX: instead of parsing this this complicatedly it should also be possible to use the Initial/FinalElementonLink!
df = df.Define("l_MC_idxs_mask", "Take(MCParticles.generatorStatus, l_MC_idxs) == 1 && abs(Take(MCParticles.PDG, l_MC_idxs)) == 11")
# Finally, convert the mask into an idx... probably not the most efficient solution over all but manual looping avoided :)
df = df.Define("l_MC_idx", "l_MC_idxs[ArgMax(l_MC_idxs_mask)]")

df = df.Define("l_MC_lvec", "ROOT::Math::PxPyPzMVector(MCParticles.momentum.x[l_MC_idx], MCParticles.momentum.y[l_MC_idx], MCParticles.momentum.z[l_MC_idx], MCParticles.mass[l_MC_idx])")

df = df.Define("l_MC_lvec_e", "l_MC_lvec.energy()")

# the same ordeal for nu, oof
df = df.Define("nu_TJ2MC_mask", "_TrueJetMCParticleLink_rec.index == nu_TJ_idx")
df = df.Define("nu_MC_idxs", "_TrueJetMCParticleLink_sim.index[nu_TJ2MC_mask]")
# urgh I accidentally had this with 11 instead of 12 and it should have not given a result but it did?!...
df = df.Define("nu_MC_idxs_mask", "Take(MCParticles.generatorStatus, nu_MC_idxs) == 1 && abs(Take(MCParticles.PDG, nu_MC_idxs)) == 12")
df = df.Define("nu_MC_idx", "nu_MC_idxs[ArgMax(nu_MC_idxs_mask)]")
df = df.Define("nu_MC_lvec", "ROOT::Math::PxPyPzMVector(MCParticles.momentum.x[nu_MC_idx], MCParticles.momentum.y[nu_MC_idx], MCParticles.momentum.z[nu_MC_idx], MCParticles.mass[nu_MC_idx])")
df = df.Define("nu_MC_lvec_e", "nu_MC_lvec.energy()")
# FIXME: for debugging
# df = df.Define("multi_qq", "Sum(qq_mask) > 1")

In [13]:
h_m_qq = df.Histo1D("m_qq")
h_m_lnu = df.Histo1D("m_lnu")

h_m_lnu_eLpL = df.Filter("beamPol_e == -1 && beamPol_p == 1").Histo1D("m_lnu")

In [14]:
h_l_e = df.Histo1D(("", ";E_{e} [GeV]", 300, 0., 150.), "l_MC_lvec_e")
h_nu_e = df.Histo1D(("", ";E_{#nu} [GeV]", 300, 0., 150.), "nu_MC_lvec_e")
h_2d_lnu_e = df.Histo2D(("", ";E_{e} [GeV];E_{#nu} [GeV]", 300, 0., 150., 300, 0., 150.), "l_MC_lvec_e", "nu_MC_lvec_e")

h_l_idx = df.Histo1D("l_MC_idx")
h_nu_idx = df.Histo1D("nu_MC_idx")
h_2d_lnu_idx = df.Histo2D(("", ";e MC idx;#nu MC idx", 50, 0., 50., 50, 0., 50.), "l_MC_idx", "nu_MC_idx")
h_2d_lnu_TJ_idx = df.Histo2D(("", ";e TJ idx;#nu TJ idx", 50, 0., 50., 50, 0., 50.), "l_TJ_idx", "nu_TJ_idx")

In [15]:
c_m_qq = ROOT.TCanvas()
h_m_qq.Draw()
c_m_qq.Draw()

c_m_lnu = ROOT.TCanvas()
h_m_lnu.Draw()
c_m_lnu.Draw()

c_m_lnu_eLpL = ROOT.TCanvas()
h_m_lnu_eLpL.Draw()
c_m_lnu_eLpL.Draw()

c_l_e = ROOT.TCanvas()
h_l_e.Draw()
c_l_e.Draw()

c_nu_e = ROOT.TCanvas()
h_nu_e.Draw()
c_nu_e.Draw()

c_2d_lnu_e = ROOT.TCanvas()
h_2d_lnu_e.Draw("colz0")
c_2d_lnu_e.Draw()

c_l_idx = ROOT.TCanvas()
h_l_idx.Draw()
c_l_idx.Draw()

c_nu_idx = ROOT.TCanvas()
h_nu_idx.Draw()
c_nu_idx.Draw()

c_2d_lnu_idx = ROOT.TCanvas()
h_2d_lnu_idx.Draw("colz0")
c_2d_lnu_idx.Draw()

c_2d_lnu_TJ_idx = ROOT.TCanvas()
h_2d_lnu_TJ_idx.Draw("colz0")
c_2d_lnu_TJ_idx.Draw()

In [16]:
# FIXME: for debug
multi_qq_evts = df.Filter("l_MC_idx > 20").Take["ULong64_t"]("rdfentry_").GetValue()
print(multi_qq_evts)

{ 2, 14, 18, 20, 31, 34, 37, 42, 46, 52, 56, 70, 73, 76, 77, 78, 82, 84, 85, 91, 96, 97, 111, 114, 124, 130, 148, 152, 168, 169, 170, 173, 174, 175, 181, 186, 188, 194, 198, 201, 202, 210, 212, 213, 214, 217, 223, 232, 247, 258, 262, 263, 264, 265, 270, 277, 283, 287, 295, 299, 303, 304, 306, 310, 318, 322, 324, 325, 334, 340, 350, 359, 361, 367, 368, 369, 374, 388, 391, 395, 400, 404, 411, 412, 416, 418, 420, 423, 431, 435, 443, 452, 459, 460, 468, 470, 475, 476, 484, 490, 492, 504, 506, 511, 513, 515, 517, 519, 528, 531, 532, 535, 548, 561, 570, 585, 591, 595, 603, 607, 612, 613, 621, 622, 625, 627, 629, 634, 639, 642, 652, 656, 657, 670, 671, 674, 681, 685, 696, 698, 699, 710, 711, 713, 722, 723, 726, 729, 738, 741, 751, 754, 756, 758, 759, 764, 794, 798, 804, 816, 818, 819, 822, 830, 837, 850, 859, 860, 862, 866, 868, 871, 876, 889, 892, 893, 896, 898, 906, 907, 910, 911, 913, 925, 943, 945, 949, 952, 953, 954, 957, 958, 962, 963, 971, 974, 978, 982, 985, 992, 993, 1005, 1007, 1017

In [17]:
df.Filter("rdfentry_ == 4").Display(("l_MC_idx", "nu_MC_idx")).Print()

+-----+----------+-----------+
| Row | l_MC_idx | nu_MC_idx | 
+-----+----------+-----------+
| 4   | 15       | 16        | 
+-----+----------+-----------+
